In [ ]:

import numpy as np
import pandas as pd
import sklearn
import nltk
from nltk import WhitespaceTokenizer


Load preprocessed data


In [ ]:
def load_tweets(file_path):
    tweets = list()
    with open(file_path, 'r', encoding='utf-8') as preprocessed_tweets:
        for tweet in preprocessed_tweets :
            tweets.append(tweet.rstrip('\n'))
    return tweets

In [ ]:
    train_pos_tweets = load_tweets('/content/processed_pos_tweets_non_transformer.txt')
    train_neg_tweets = load_tweets('/content/processed_neg_tweets_non_transformer.txt')
    test_tweets = load_tweets('/content/processed_test_tweets_non_transformer.txt')
    print("Tweets loaded")

Tweets loaded


Convert to Dataframe

In [ ]:
    from sklearn.model_selection import train_test_split
    from sklearn.utils import shuffle
    seed =12222
    train_neg_labels = [0] * len(train_neg_tweets)
    train_pos_labels = [1] * len(train_pos_tweets)

    train_tweets = train_pos_tweets + train_neg_tweets
    train_labels = train_pos_labels + train_neg_labels
    #Shuffle
    train_tweets, train_labels = shuffle(train_tweets, train_labels, random_state=10)
    data = pd.DataFrame({'tweet': train_tweets, 'label': train_labels})

    X = list(data["tweet"])
    y = list(data["label"])
    # X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.05,stratify=y,random_state=seed)
    # X_train_set = pd.DataFrame({'tweet': X_train, 'label': y_train})
    # X_val_set = pd.DataFrame({'tweet': X_val, 'label': y_val})

    # X_test_set = pd.DataFrame({'tweet': test_tweets})


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split



In [ ]:
tf.random.set_seed(seed)

#Load Data
tweets = np.array(X)
labels = np.array(y)
X_train_set = pd.DataFrame(columns = ['tweet', 'label'], data=np.array([tweets, labels]).T)
#Tokenize tweets and pad
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(X_train_set['tweet'].values)
X = tokenizer.texts_to_sequences(X_train_set['tweet'].values)
X = pad_sequences(X, maxlen = 100)

#Define Model
embed_dim = 500 #400
max_feat = 15000 #10000

#Define Bidirectional LSTM layer
LSTMLayer = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(200, dropout=0.3, recurrent_dropout=0.25))

#Simple LSTM model (Noticed that the stacked one would perform worse)
model = Sequential()

model.add(Embedding(max_feat, embed_dim, input_length = X.shape[1]))

model.add(SpatialDropout1D(0.35))

model.add(LSTMLayer)

model.add(Dense(2,activation='softmax'))

model.compile(optimizer='adam',loss = 'categorical_crossentropy',metrics = ['accuracy'])
print(model.summary())



#Create Validation set
Y = pd.get_dummies(X_train_set['label']).values
X_train, X_val, Y_train, Y_val = train_test_split(X,Y, test_size = 0.05, random_state = seed)

#Train Model
batch_size = 128
history = model.fit(X_train, Y_train,  validation_data = [X_val, Y_val], epochs = 25,
                    steps_per_epoch = 300, batch_size = batch_size, verbose = 1)


Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_20 (Embedding)    (None, 100, 500)          7500000   
                                                                 
 spatial_dropout1d_20 (Spati  (None, 100, 500)         0         
 alDropout1D)                                                    
                                                                 
 bidirectional_16 (Bidirecti  (None, 400)              1121600   
 onal)                                                           
                                                                 
 dense_16 (Dense)            (None, 2)                 802       
                                                                 
Total params: 8,622,402
Trainable params: 8,622,402
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/25
300/300 [==================

In [ ]:
# test_tweets
X_test = tokenizer.texts_to_sequences(test_tweets)
X_test = pad_sequences(X_test, maxlen = 100)
y_test = model.predict(X_val,batch_size=len(X_val))
y_preds = [-1 if np.argmax(val) == 0 else 1 for val in y_test]
df = pd.DataFrame(y_preds, columns=["Prediction"])
df.index.name = "Id"
df.index += 1
df.to_csv("/content/drive/MyDrive/test_data_word2vec_final.csv")

1/1 [==============================] - 1s 585ms/step
